<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/01_musik_der_gro%C3%9Fst%C3%A4dte/musik_der_gro%C3%9Fst%C3%A4dte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Musik der Großstädte

Der Musikdienst lieferte Daten über die von Nutzern in Moskau und St. Petersburg abgehörten Tracks für einen bestimmten Zeitraum.
Es ist notwendig, Fragen zu beantworten und Hypthesen zu überprüfen:
- musik in zwei Städten — Moskau und St. Petersburg - hören Sie in verschiedenen Modi;
- die Listen der zehn beliebtesten Genres am Montagmorgen und am Freitagabend haben charakteristische Unterschiede;
- die Bewohner der beiden Städte bevorzugen verschiedene Musikgenres.

In [1]:
# Wir importieren die notwendigen Bibliotheken.
import pandas as pd

In [2]:
# Wir laden Tabellen mit Daten.
df = pd.read_csv('/content/music_project.csv')

In [4]:
# Wir werden die erhaltenen Daten studieren
df.info()
display(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


(65079, 7)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


**Schlussfolgerungen**

Jede Zeile der Tabelle enthält Informationen zu Songs eines bestimmten Genres in einer bestimmten Performance, die Benutzer zu bestimmten Zeiten und Wochentagen in einer der Städte angehört haben. Zwei Probleme, die angegangen werden müssen: fehlende und schlechte Spaltennamen. Die Spalten time, day und City sind besonders wertvoll, um Arbeitshypothesen zu testen. Mit den Daten aus der Spalte genre können Sie die beliebtesten Genres kennenlernen.

# Datenvorverarbeitung

In [5]:
# Wir erhalten eine Liste der Spaltennamen.
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [7]:
# Wir werden die Spalten umbenennen, um die weitere Arbeit zu erleichtern.
df.set_axis(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'], 
            axis = 'columns',inplace = True)

# Mal sehen was passiert
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

In [8]:
# Wir werden prüfen, ob es Nullwerte gibt.
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [10]:
# Wir ersetzen die fehlenden Werte in den Spalten mit dem Titel und dem Interpreten durch die Zeile 'unknown'.
df['artist_name']=df['artist_name'].fillna('unknown')
df['track_name']=df['track_name'].fillna('unknown')

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [11]:
# Wir entfernen leere Werte in der Spalte mit den Genres.
df.dropna(subset = ['genre_name'], inplace = True)

In [12]:
# Mal sehen was passiert
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

In [13]:
# Wir werden prüfen, ob es Duplikate gibt.
df.duplicated().sum()

3755

In [14]:
# Wir werden Duplikate entfernen.
df = df.drop_duplicates().reset_index(drop=True)

# Mal sehen was passiert
df.duplicated().sum()

0

In [16]:
# Wir speichern die Liste der eindeutigen Werte einer Spalte mit Genres in der Variablen genres_list.
genres_list = df['genre_name'].unique() 
genres_list

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'alternative', 'children', 'rnb', 'hip', 'jazz',
       'postrock', 'latin', 'classical', 'metal', 'reggae', 'tatar',
       'blues', 'instrumental', 'rusrock', 'dnb', 'türk', 'post',
       'country', 'psychedelic', 'conjazz', 'indie', 'posthardcore',
       'local', 'avantgarde', 'punk', 'videogame', 'techno', 'house',
       'christmas', 'melodic', 'caucasian', 'reggaeton', 'soundtrack',
       'singer', 'ska', 'shanson', 'ambient', 'film', 'western', 'rap',
       'beats', "hard'n'heavy", 'progmetal', 'minimal', 'contemporary',
       'new', 'soul', 'holiday', 'german', 'tropical', 'fairytail',
       'spiritual', 'urban', 'gospel', 'nujazz', 'folkmetal', 'trance',
       'miscellaneous', 'anime', 'hardcore', 'progressive', 'chanson',
       'numetal', 'vocal', 'estrada', 'russian', 'classicmetal',
       'dubstep', 'club', 'deep', 'southern', 'black', 'folkrock',
       'fitness', 'french', 'd

In [17]:
# Wir deklarieren die Funktion find_genre(), um nach impliziten Duplikaten in einer Spalte mit Genres zu suchen. 
# Zum Beispiel, wenn der Name des gleichen Genres in verschiedenen Wörtern geschrieben ist.
def find_genre(genre_search):
    count_genre = 0
    for genre in genres_list:
        if genre == genre_search:
            count_genre += 1
    return count_genre

In [22]:
# Wir rufen die Funktion find_genre() auf, um nach verschiedenen Varianten 
# des Hip-Hop-Genres in der Tabelle zu suchen.
find_genre('hip')

1

In [19]:
find_genre('hop')

0

In [20]:
find_genre('hip-hop')

0

In [21]:
find_genre('hiphop')

1

In [23]:
# Wir deklarieren die Funktion find_hip_hop(), die den falschen Namen dieses Genres 
# in der Spalte 'genre_name' durch 'hip hop' ersetzt und überprüft, ob die Ersetzung erfolgreich war.
def find_hip_hop(df, wrong):
    df['genre_name'] = df['genre_name'].replace(wrong,'hiphop')
    final = df[df['genre_name'] == wrong]['genre_name'].count()
    return final

In [24]:
# Wir rufen die Funktion find_hip_hop() für unsere Tabelle auf
find_hip_hop(df, 'hip')

0

In [25]:
# Mal sehen was passiert
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   city         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


**Schlussfolgerungen**

In der Vorverarbeitungsphase wurden Lücken, Probleme mit Spaltennamen und Duplikate in den Daten festgestellt. Die Lücken sind ausgefüllt, die Duplikate wurden entfernt, die Spaltennamen sind jetzt korrekt und bequem für die weitere Arbeit.
Wir haben nicht begonnen, alle Zeilen mit fehlenden Werten zu löschen, da sie die Namen von Genres enthalten, die uns für die Analyse wichtig sind.